In [1]:
import os
import numpy as np

from skimage.transform import resize
from skimage.io import imsave

from keras.models import Model
from keras.layers import Input, concatenate
from keras.layers.core import Layer 
from keras.layers.core import Activation, Lambda, Reshape, Permute
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras import models

from keras import backend as K

/home/sara/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 480
img_cols = 480

smooth = 1.

In [3]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * i360ntersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

In [4]:
import keras
from keras.models import Model
from keras.layers import Activation, BatchNormalization, Dense, Input
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras import backend as K

k_size=3

def custom_loss(y_true, y_pred):
    y_t = K.reshape(y_true,[-1,1])
    y_p = K.reshape(y_pred,[-1,4])
    losses = K.sparse_categorical_crossentropy(y_p,y_t, from_logits=True)
    return K.sum(losses)

def SegNet():
    inputs = Input([480,480,1])

    # Encoder
    e = Convolution2D(32,k_size,k_size,border_mode='same')(inputs)
    e = BatchNormalization()(e)
    e = Activation('relu')(e)
    e = MaxPooling2D()(e)

    e = Convolution2D(32,k_size,k_size,border_mode='same')(e)
    e = BatchNormalization()(e)
    e = Activation('relu')(e)
    e = MaxPooling2D()(e)

    e = Convolution2D(32,k_size,k_size,border_mode='same')(e)
    e = BatchNormalization()(e)
    e = Activation('relu')(e)
    e = MaxPooling2D()(e)

    # Decoder
    d = UpSampling2D()(e)
    d = Convolution2D(32,k_size,k_size,border_mode='same')(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)

    d = UpSampling2D()(d)
    d = Convolution2D(32,k_size,k_size,border_mode='same')(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)

    d = UpSampling2D()(d)
    d = Convolution2D(32,k_size,k_size,border_mode='same')(d)
    d = BatchNormalization()(d)
    d = Activation('sigmoid')(d)

    out = Convolution2D(1,1,1)(d)

    model = Model(inputs, out)
    model.summary()
    model.compile(optimizer='adam', loss=custom_loss)

    return model

In [5]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, 1), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

        #     imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [6]:
def load_train_data():
    imgs_train = np.load('npy-with/preprocess/train-images.npy')
    imgs_mask_train = np.load('npy-with/preprocess/train-masks.npy')
    return imgs_train, imgs_mask_train

def load_validation_data():
    imgs_validation = np.load('npy-with/preprocess/validation-images.npy')
    imgs_mask_validation = np.load('npy-with/preprocess/validation-masks.npy')
    return imgs_validation, imgs_mask_validation

In [ ]:
print('-'*30)
print('Loading and preprocessing train data...')
print('-'*30)
imgs_train, imgs_mask_train = load_train_data()
print(len(imgs_train))
imgs_train = preprocess(imgs_train)
imgs_mask_train = preprocess(imgs_mask_train)

imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255.  # scale masks to [0, 1]

In [ ]:
print('-'*30)
print('Loading and preprocessing validation data...')
print('-'*30)
imgs_validation, imgs_mask_validation = load_validation_data()
print(len(imgs_validation))
imgs_validation = preprocess(imgs_validation)
imgs_mask_validation = preprocess(imgs_mask_validation)

imgs_validation = imgs_validation.astype('float32')
mean = np.mean(imgs_validation)  # mean for data centering
std = np.std(imgs_validation)  # std for data normalization

imgs_validation -= mean
imgs_validation /= std

imgs_mask_validation = imgs_mask_validation.astype('float32')
imgs_mask_validation /= 255.  # scale masks to [0, 1]

In [ ]:
print('-'*30)
print('Creating and compiling model...')
print('-'*30)

model = SegNet()

model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),loss=dice_coef_loss, metrics=[dice_coef, 'acc',Jaccard_index, Specificity, Sensitivity])
model_checkpoint = ModelCheckpoint('SegNet-with-weights.h5', monitor='val_loss', save_best_only=True)

In [ ]:
print('-'*30)
print('Fitting model...')
print('-'*30)
history = model.fit(imgs_train, imgs_mask_train, batch_size=10, epochs=50, verbose=2, shuffle=True,
          validation_data=(imgs_validation, imgs_mask_validation),
          callbacks=[model_checkpoint])